# Table of Contents
<p>
<div class="lev1"><a href="#Data-from-the-Web"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data from the Web</a></div>
<div class="lev1"><a href="#Getting-the-data"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Getting the data</a></div>
<div class="lev2"><a href="#Requesting-ISA-form"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Requesting ISA form</a></div>
<div class="lev2"><a href="#Finding-form-IDs"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Finding form IDs</a></div>
<div class="lev2"><a href="#Filtering-and-getting-the-data"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Filtering and getting the data</a></div>



# Data from the Web

In this homework we will extract interesting information from IS-Academia, the educational portal of EPFL. Specifically, we will focus on the part that allows public access to academic data. The list of registered students by section and semester is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need. On this form you can select the data to download based on different criteria (e.g., year, semester, etc.)

You are not allowed to download manually all the tables -- rather you have to understand what parameters the server accepts, and generate accordingly the HTTP requests. For this task, Postman with the Interceptor extension can help you greatly. I recommend you to watch this brief tutorial to understand quickly how to use it. Your code in the iPython Notebook should not contain any hardcoded URL. To fetch the content from the IS-Academia server, you can use the Requests library with a Base URL, but all the other form parameters should be extracted from the HTML with BeautifulSoup. You can choose to download Excel or HTML files -- they both have pros and cons, as you will find out after a quick check. You can also choose to download data at different granularities (e.g., per semester, per year, etc.) but I recommend you not to download all the data in one shot because 1) the requests are likely to timeout and 2) we will overload the IS-Academia server.


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import requests
import re
from bs4 import BeautifulSoup
sns.set_context('notebook')

# Getting the data

## Finding ISA form 

The first part of the job in order to get the data is to get the parameters required to get the data we want.

In this purpose, we first do a get request on the ISA form with the link <http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247>.

We also use BeautifulSoup on the resulting html response in order to parse it later.

In [3]:
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')
r.headers['content-type']
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

## Finding form IDs

Now that we've got the form's html code, we need to know which values of the form are used to filter and displayed the desired data. The values we're interested in are 'unité académique', 'période académique' and 'période pédagogique' (corresponding respectively to section, academic year and semester).

By inspecting the html code, we saw that the form items are 'option', it is then easy to get their value by using BeautifupSoup find and find_all method.

The following code will simply find the option value corresponding to section 'Informatique', and output it's value (the id used to filter the result).
```python
    soup.find('option', text = re.compile('Informatique'))['value']
```

We do the same thing for Bachelor 1st and 6th semester.
```python
    bachelor_1_id = soup.find('option', text = re.compile('Bachelor semestre 1'))['value']
    bachelor_6_id = soup.find('option', text = re.compile('Bachelor semestre 6'))['value']
```

And we get the academic years ids from 2007-2008 to 2016-2017 using a for loop (see in the cell below)

In [4]:
soup.find_all("option")

informatique_id = soup.find('option', text = re.compile('Informatique'))['value']
print("Id of informatique : ", informatique_id, "\n")

bachelor_tags = soup.find_all('option', text = re.compile('Bachelor'))
bachelor_1_id = soup.find('option', text = re.compile('Bachelor semestre 1'))['value']
bachelor_6_id = soup.find('option', text = re.compile('Bachelor semestre 6'))['value']
print("Id of Bachelor semester 1: ", bachelor_1_id,"\n")
print("Id of Bachelor semester 6: ", bachelor_6_id,"\n")

year_tags = soup.find_all('option', text = re.compile('20'))
year_ids = []
for y in range(2007, 2017):
    school_year = str(y) + "-" + str(y+1)
    year_ids += [soup.find('option', text = re.compile(school_year))['value']]
    
print("years ids : (from 2007-2008 to 2016-2017)", year_ids)




Id of informatique :  249847 

Id of Bachelor semester 1:  249108 

Id of Bachelor semester 6:  942175 

years ids : (from 2007-2008 to 2016-2017) ['978181', '978187', '978195', '39486325', '123455150', '123456101', '213637754', '213637922', '213638028', '355925344']


## Filtering and getting the data

Now that we know the interesting IDs used in the form, we need to filter and request our data. For this purpose, we used Postman and Postman interceptor to intercept and inspect the request method used to get the data from the formula. 
  
</br>




The picture below shows all parameters used in the URL to filter and return results for:
* Section "Informatique"
* Academic period "2016-2017"
* Pedagogic period "Bachelor semestre 1"

<p>
    <img src="img/postman.png" alt="postman" align="center"/>
</p>

After playing a bit with the URL, we conclude that not all parameters were mandatory, the required parameters and their values are:

|parameter  | value |
|-----------|-------|
|ww_b_list  |must be '1'|  
|ww_i_reportmodel|must be '133685247'|
|ww_i_reportModelXsl|must be '133685270'|
|ww_x_UNITE_ACAD|correspond to the id of the section, taken from the form|
|ww_x_PERIODE_ACAD|correspond to the id of the academic year, taken from the form|
|ww_x_PERIODE_PEDAGO|correspond to the id of the semester, taken from the form|



In [37]:
params = {'ww_b_list':'1',
        'ww_i_reportmodel':'133685247',
        'ww_i_reportModelXsl':'133685270',
        'ww_x_UNITE_ACAD':informatique_id,
        'ww_x_PERIODE_ACAD':year_ids[2012-2007],
        'ww_x_PERIODE_PEDAGO':bachelor_1_id}
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?', params)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')


In [38]:
ww_x_GPS = soup.find_all('a')[1].get('onclick').split("ww_x_GPS=")[1].split("')")[0]
ww_x_GPS

'1378362092'

In [39]:
params['ww_x_GPS'] = ww_x_GPS
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?', params)
r.headers['content-type']

'text/html; charset=ISO-8859-15'

In [40]:
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

In [41]:
students_tr = soup.body.hr.table.find_all('tr')[2:]
students = []
for i in range (0,len(students_tr)):
    student = students_tr[i].find_all('td')
    students.append([student[0].text,student[1].text.replace(u'\xa0', u' '),student[7].text,student[10].text])

pd_student = pd.DataFrame(students, columns=['A', 'B', 'C', 'D'])
pd_student



,A,B,C,D
0,Monsieur,Albrecht Pablo,Présent,212726
1,Monsieur,Alvard Edouard,Présent,223371
2,Madame,Ammann Gaëlle,Présent,228116
3,Monsieur,Amorim Afonso Caldeira Da Silva Pedro Maria,Présent,213618
4,Monsieur,Amrani Kamil,Présent,226305
5,Monsieur,Andreina Sébastien Laurent,Présent,215623
6,Monsieur,Angerand Grégoire Georges Jacques,Présent,212464
7,Monsieur,Antunes Nelson Tiago,Présent,224198
8,Monsieur,Armand Alexis Kevin,Présent,224617
9,Monsieur,Aulbach Adrian,Présent,214964
